In [ ]:
import yfinance as yf
# Define the ticker symbol for the stock you're interested in
ticker_symbol = "AAPL" # Example: Apple Inc.

# Download historical stock price data
stock_data = yf.download(ticker_symbol, start="2020-01-01", end="2022- 01-01")
# Save the data to a CSV file
stock_data.to_csv("stock_prices.csv")

[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAPL']: ValueError("time data '2022- 01-01' does not match format '%Y-%m-%d'")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from sklearn.metrics import mean_squared_error

# Load data
data = pd.read_csv('stock_prices.csv')
# Preprocess data
prices = data['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
prices_scaled = scaler.fit_transform(prices)

# Define function to create input sequences and corresponding labels
def create_sequences(data, seq_length):
  X, y = [], []
  for i in range(len(data) - seq_length): X.append(data[i:(i + seq_length)])
  y.append(data[i + seq_length])
  return np.array(X), np.array(y)
# Choose sequence length and split data into sequences and labels
sequence_length = 10
X, y = create_sequences(prices_scaled, sequence_length)
# Split data into training and testing sets
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size
X_train, X_test = X[0:train_size], X[train_size:len(X)]
y_train, y_test = y[0:train_size], y[train_size:len(y)]

# Define and train the RNN model
model = Sequential([SimpleRNN(50, input_shape=(sequence_length, 1)), Dense(1)])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=2)

# Evaluate the model
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverse scaling for predictions
train_predict = scaler.inverse_transform(train_predict)
y_train_inv = scaler.inverse_transform(y_train)
test_predict = scaler.inverse_transform(test_predict)
y_test_inv = scaler.inverse_transform(y_test)

# Calculate root mean squared error
train_score = np.sqrt(mean_squared_error(y_train_inv, train_predict[:,0]))
print('Train Score: %.2f RMSE' % (train_score))
test_score = np.sqrt(mean_squared_error(y_test_inv, test_predict[:,0]))
print('Test Score: %.2f RMSE' % (test_score))

# Plotting
plt.plot(np.arange(0, len(y_train_inv)), y_train_inv, 'g', label="actual")
plt.plot(np.arange(0, len(train_predict)), train_predict[:,0], 'r', label="predicted")
plt.legend(loc='upper left')
plt.title("Training Set")
plt.show()

plt.plot(np.arange(len(y_train_inv), len(y_train_inv) + len(y_test_inv)), y_test_inv, 'g', label="actual")
plt.plot(np.arange(len(y_train_inv), len(y_train_inv) + len(y_test_inv)), test_predict[:,0], 'r', label="predicted")
plt.legend(loc='upper left')
plt.title("Test Set")
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'stock_prices.csv'